In [13]:
import numpy as np
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys

from missing_process.block_rules import *
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from tqdm import tqdm


In [12]:
def load_origin_data(missingtypelist,datalist_name):
    '''
    create density plot
    '''
    

    if datalist_name == "real":
        datalist = real_datalist
    elif datalist_name == "syn":
        datalist = syn_datalist

    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")

        # load data and its mask
        for dataname in datalist:
            directory_path = f"../datasets/{dataname}"    
            norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
            
            print(dataname)


            for rule_name in missing_rule:
                observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')


            return norm_values,observed_masks

In [11]:
def load_impute_data(missingtype,model_name,rule_name,dataname,fold,seed = 1):

    train_impute = np.load(f'../impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_train.npy')
    test_impute = np.load(f'../impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_test.npy')
    return train_impute,test_impute

def load_train_test(index_file,norm_values,observed_masks):  

    train_index = index_file["train_index"]
    test_index = index_file["test_index"]

    train_values = norm_values[train_index,:]

    train_masks = observed_masks[train_index,:]

    test_values = norm_values[test_index,:]

    test_masks = observed_masks[test_index,:]

    return train_values,train_masks,test_values,test_masks

In [1]:
# datalist = real_datalist

# plot = True

# missingtypelist = ["logistic","diffuse","quantile"]
# missingtypelist = ["logistic"]
# #model_list = ["mean","knn","gain","XGB","mice","mf","notmiwae","miwae","missforest","hyper","tabcsdi"]
# model_list = ["gain","notmiwae","miwae","tabcsdi","ot"]
# for model_name in model_list:
#        cal_rmse_plot(missingtypelist,model_name,datalist)      
            
        


In [10]:
def cal_res_plot(missingtypelist,model_name,datalist):
    quantilelist = ["Q1_Q2_0.25","Q1_Q2_0.75", #adjcent
                    "Q1_Q4_0.25","Q1_Q4_0.75", #extrme
                    "Q2_Q4_0.25","Q2_Q4_0.75"] # seperated
    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")

        # load data and its mask
        for dataname in tqdm(datalist):
            directory_path = f"../datasets/{dataname}"    
            norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
            f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
            index_file = json.load(f)
            

            for rule_name in tqdm(missing_rule):
                if (rule_name in ["0.5"] and missingtype == "diffuse") or (rule_name in ["0.5"] and missingtype == "logistic") or (rule_name in quantilelist and missingtype == "quantile"):
                    observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')

                    for fold in index_file:
                        index = index_file[fold]
                        train_values,train_masks,test_values,test_masks = load_train_test(index,norm_values,observed_masks)

                        impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)
                        try:
                            
                            rmse = RMSE_columnwise(test_masks,test_values,impute_test)
                            
                            make_line_plot(test_values,impute_test,test_masks,title = f"{dataname} {missingtype} {rule_name} {model_name} ",subtitle=rmse)

                        except:
                            pass
                        break

        

In [9]:
real_datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

#missingtypelist = ["quantile","diffuse","logistic"]
missingtypelist = ["logistic"]

seed = 1
nfold = 5



In [8]:
def load_origin_data(missingtypelist,datalist_name):
    '''
    create density plot
    '''
    

    if datalist_name == "real":
        datalist = real_datalist
    elif datalist_name == "syn":
        datalist = syn_datalist

    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")

        # load data and its mask
        for dataname in datalist:
            directory_path = f"../datasets/{dataname}"    
            norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
            
            print(dataname)


            for rule_name in missing_rule:
                observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')


            return norm_values,observed_masks

In [7]:
def RMSE(mask,complete,imputed):
    missing_data_points = mask == 0

    # Extracting the originally missing data from both complete and imputed data
    original_values = complete[missing_data_points]
    imputed_values = imputed[missing_data_points]

    # Calculating RMSE
    rmse = np.sqrt(np.mean((original_values - imputed_values) ** 2))

    return rmse

def residual(mask,complete,imputed):
    missing_data_points = mask == 0

    # Extracting the originally missing data from both complete and imputed data
    original_values = complete[missing_data_points]
    imputed_values = imputed[missing_data_points]

    # Calculating RMSE
    res = original_values - imputed_values

    return res


def complete_computed_value(mask, complete, imputed):
    # Create an empty array with the same shape as 'complete' and 'imputed'
    computed = np.empty_like(complete)

    # Fill in the values
    computed[mask == 1] = complete[mask == 1]  # Use values from 'complete' where mask is 1
    computed[mask == 0] = imputed[mask == 0]   # Use values from 'imputed' where mask is 0

    return computed

def complete_computed_value(mask, complete, imputed):
    # Create an array filled with 'nan' with the same shape as 'complete' and 'imputed'
    computed = np.full_like(complete, np.nan, dtype=float)

    # Fill in the imputed values where mask is 0
    computed[mask == 0] = imputed[mask == 0]

    return computed

# Example usage
# mask, complete, and imputed should be numpy arrays of the same shape
# complete_computed = complete_computed_value(mask, complete, imputed)


def MAE(mask,complete,imputed):
    missing_data_points = mask == 0

    # Extracting the originally missing data from both complete and imputed data
    original_values = complete[missing_data_points]
    imputed_values = imputed[missing_data_points]

    # Calculating RMSE
    mae = np.mean(np.abs(original_values - imputed_values))

    return mae

In [6]:
def load_impute_data(missingtype,model_name,rule_name,dataname,fold,seed = 1):

    train_impute = np.load(f'../impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_train.npy')
    test_impute = np.load(f'../impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_test.npy')
    return train_impute,test_impute

def load_train_test(index_file,norm_values,observed_masks):  

    train_index = index_file["train_index"]
    test_index = index_file["test_index"]

    train_values = norm_values[train_index,:]

    train_masks = observed_masks[train_index,:]

    test_values = norm_values[test_index,:]

    test_masks = observed_masks[test_index,:]

    return train_values,train_masks,test_values,test_masks

In [5]:
def cal_rmse_plot(missingtypelist,model_name,datalist):
    quantilelist = ["Q1_Q2_0.5","Q1_Q4_0.5","Q2_Q4_0.5","Q1_Q2_1.0","Q1_Q4_1.0","Q2_Q4_1.0"]
    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")

        # load data and its mask
        for dataname in tqdm(datalist):
            directory_path = f"../datasets/{dataname}"    
            norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
            f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
            index_file = json.load(f)
            

            for rule_name in tqdm(missing_rule):
                if (rule_name in ["0.8"] and missingtype == "diffuse") or (rule_name in ["0.5","0.8"] and missingtype == "logistic") or (rule_name in quantilelist and missingtype == "quantile"):
                    observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')

                    for fold in index_file:
                        index = index_file[fold]
                        train_values,train_masks,test_values,test_masks = load_train_test(index,norm_values,observed_masks)
                        
                        impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)

                        train_rse = residual(train_masks,train_values,impute_train)
                        test_rse = residual(test_masks,test_values,impute_test)
                        
                        rse = RES_columnwise(train_masks,train_values,impute_train)
                        make_plot(test_values,impute_test,test_masks,title = f"{dataname} {missingtype} {rule_name} {model_name} ",subtitle=rse)




In [4]:
def RES_columnwise(mask, complete, imputed):


    # Initializing an array to store RMSE for each column
    rse_per_column = np.zeros(complete.shape[1])

    # Iterate over each column
    for i in range(complete.shape[1]):
        missing_data_points = mask[:, i] == 0
        # Extracting the originally missing data from both complete and imputed data
        original_values = np.reshape(complete[missing_data_points, i],-1)
        imputed_values = np.reshape(imputed[missing_data_points, i],-1)

        rse_per_column[i] = np.mean((original_values - imputed_values))
    return np.nan_to_num(rse_per_column, nan=0)

In [19]:
def make_plot(norm_value,impute,mask, title = None,subtitle = None):

    indices = np.argsort(subtitle)[-8:]

    indices = indices if len(indices) < 8 else list(range(8))

    # Sorting the indices and converting to a list
    sorted_indices = np.sort(indices).tolist()
        # Create a 4x2 grid of subplots
    fig, axes = plt.subplots(4, 2, figsize=(12, 12))
    axes = axes.ravel()  # Flatten the 4x2 grid for easy iteration

    impute_value = complete_computed_value(mask, norm_value, impute)
    true_value = complete_computed_value(mask, norm_value, norm_value)
    for i in range(len(sorted_indices)):

        norm_column = norm_value[:, sorted_indices[i]]
        impute_column = impute_value[:, sorted_indices[i]]
        true_column = true_value[:, sorted_indices[i]]


        sns.residplot(x=impute_column, y=true_column, lowess=True, color="g",ax=axes[i])

        axes[i].set_xlabel('Imputed Values')
        axes[i].set_ylabel('Residuals')
        if subtitle[i]:
            axes[i].set_title(f'Column {sorted_indices[i]+1} Residual: {round(subtitle[sorted_indices][i],3)}')
        else:
            axes[i].set_title(f'Fully Observed Column {sorted_indices[i]+1}')
        #axes[i].legend()

    if title is not None:
        fig.suptitle(title, fontsize=16)
        # Adjust the layout
    plt.tight_layout()
    plt.savefig(f"../plot/residual/{title}.png")
    #plt.show()

In [2]:
# datalist = real_datalist

# plot = True

# missingtypelist = ["logistic","diffuse","quantile"]
# missingtypelist = ["logistic"]
# model_list = ["mean","knn","gain","XGB","mice","mf","notmiwae","miwae","missforest","hyper","tabcsdi"]
# model_list = ["ot","miwae","notmiwae","gain","tabcsdi","gain"]
# for model_name in tqdm(model_list):
#        overall_res_plot(missingtypelist,model_name,datalist)      
            
        

In [2]:
def overall_res_plot(missingtypelist,model_name,datalist):
    quantilelist = ["Q1_Q2_0.5","Q1_Q4_0.5","Q2_Q4_0.5","Q1_Q2_1.0","Q1_Q4_1.0","Q2_Q4_1.0"]
    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")

        # load data and its mask
        for dataname in tqdm(datalist):
            directory_path = f"../datasets/{dataname}"    
            norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
            f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
            index_file = json.load(f)
            
            rule_name_list = []
            residual_list = [[],[],[]]
            for rule_name in tqdm(missing_rule):
 
                if ((rule_name in ["0.2","0.4","0.6","0.8"] and missingtype == "diffuse") or 
                (rule_name in ["0.2","0.4","0.6","0.8"] and missingtype == "logistic") or 
                (rule_name in quantilelist and missingtype == "quantile")):

                    observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')
                    rule_name_list.append(rule_name)

                    for fold in index_file:
                        index = index_file[fold]
                        train_values,train_masks,test_values,test_masks = load_train_test(index,norm_values,observed_masks)
                        
                        impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)

                        impute_value = complete_computed_value(test_masks, test_values, impute_test)
                        true_value = complete_computed_value(test_masks, test_values, test_values)
                        res = impute_value - true_value
                        residual_list[0].append(true_value)
                        residual_list[1].append(impute_value)
                        residual_list[2].append(res)
                        break



            fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
            axes = axes.flatten()  # Flattening the 2D array of axes for easier indexing

            # Looping through each pair of residuals to plot
            for i in range(len(residual_list[0])):
                # Using Seaborn's residplot for each subplot
                true_values, impute_values = np.reshape(residual_list[0][i],-1),np.reshape(residual_list[1][i],-1)
                sns.residplot(x=impute_values, y=true_values, lowess=True, color="g", ax=axes[i])
                axes[i].set_xlabel('Imputed Values')
                axes[i].set_ylabel('Residuals')
                axes[i].set_title(f'Parameter: {rule_name_list[i]} - Avg Residual = {np.nanmean(residual_list[2][i])}')
            fig.suptitle(f'{missingtype} - {dataname} {model_name}', fontsize=16)
            plt.tight_layout()
            plt.savefig(f"../plot/residual_overall/{missingtype} {dataname} {model_name}.png")
            #plt.show()
            
